<a href="https://colab.research.google.com/github/aaronayitey/Sentiment-Analysis-Covid_19-Twitter-Reactions/blob/main/Model_ROBERTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from google.colab import drive
drive.mount('/content/drive')

In [39]:
!pip install huggingface_hub transformers datasets gradio pipreqs TextBlob emot xformers transformers[torch] accelerate -U

In [40]:
!pip install typing-extensions<4.6.0

In [41]:
!pip install datasets huggingface_hub -q

In [42]:
# Import key libraries and packages

import numpy as np
import os
import uuid
import pandas as pd
import re

from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TrainingArguments, Trainer
from transformers import IntervalStrategy
from transformers import EarlyStoppingCallback
from transformers import pipeline
from transformers import TrainingArguments
from transformers import Trainer
from torch import nn
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from sklearn.metrics import accuracy_score

In [43]:
notebook_login()

In [44]:

# Disable Weights & Biases
os.environ["WANDB_DISABLED"] = "true"

In [152]:
# Load the CSV file into a DataFrame

url = "https://github.com/Azubi-Africa/Career_Accelerator_P5-NLP/raw/master/zindi_challenge/data/Train.csv"

df_train = pd.read_csv(url)

In [153]:
df_train.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [154]:
# Select rows with missing values
df_train.isnull().sum()

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64

In [155]:
# Select rows with missing values
df_train[df_train.isnull().any(axis=1)]

,tweet_id,safe_text,label,agreement
4798,RQMQ0L2A,#lawandorderSVU,NaN,NaN
4799,I cannot believe in this day and age some pare...,1,0.666667,NaN


In [156]:
# Extract complete text from 'safe_text' column
complete_text = df_train.iloc[4798]['safe_text']
complete_text

'#lawandorderSVU '

In [157]:
# Select row by index and assign values to columns
df_train.loc[4798, 'label'] = 0
df_train.loc[4798, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df_train.iloc[4798, df_train.columns.get_loc('safe_text')] = complete_text

In [158]:
# Generate random UUID string for tweet_id
'''UUIDs are often used in software applications for various purposes such as generating unique IDs for entities,
tracking unique user sessions, or creating unique file names'''
rand_tweet_id = str(uuid.uuid4())

# Select row by index and assign values to columns
row_index = 4799
df_train.loc[row_index, 'tweet_id'] = rand_tweet_id
df_train.loc[row_index, 'label'] = 1
df_train.loc[row_index, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df_train.iloc[row_index, df_train.columns.get_loc('safe_text')] = df_train.iloc[row_index, 1]

# Clean Tweets

In [122]:
def clean_text(text):
    # Remove usernames
    text = remove_usernames(text)

    # Remove hashtags
    text = remove_hashtags(text)

    # Remove URLs
    text = remove_urls(text)

    # Remove special characters
    text = remove_special_chars(text)

    return text

def remove_usernames(text):
    pattern = r'@\w+\s?'
    result = re.sub(pattern, '', text)
    return result

def remove_hashtags(text):
    pattern = r'#\w+\s?'
    result = re.sub(pattern, '', text)
    return result

def remove_urls(string):
    pattern = r'https?://\S+'
    result = re.sub(pattern, '', string)
    return result

def remove_special_chars(text):
    pattern = r'[*\n]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

In [123]:
df_train['safe_text'] = df_train['safe_text'].apply(clean_text)

In [159]:
df_train['label'].value_counts()

 0.0    4909
 1.0    4054
-1.0    1038
Name: label, dtype: int64

In [160]:
# Find the maximum count among all classes
max_class_count = df_train['label'].value_counts().max()

# Group the dataframe by 'label'
grouped = df_train.groupby('label')

# Sample each group to match the max_class_count
balanced_train = grouped.apply(lambda x: x.sample(max_class_count, replace=True)).reset_index(drop=True)

# The 'balanced_df' now contains an equal number of instances for each class
balanced_train['label'].value_counts()

-1.0    4909
 0.0    4909
 1.0    4909
Name: label, dtype: int64

In [161]:
# Split the train data => {train, eval}
train, eval = train_test_split(balanced_train, test_size=0.2, random_state=42)

In [162]:
train.info()

In [163]:
eval.info()

In [164]:
file_path = '/content/drive/MyDrive/NLP/data_sent'
train.to_csv(os.path.join(file_path, "train_subset.csv"), index=False)
eval.to_csv(os.path.join(file_path, "eval_subset.csv"), index=False)

In [165]:
# Load the CSV files into a dataset

dataset = load_dataset('csv', data_files={
    'train': '/content/drive/MyDrive/NLP/data_sent/train_subset.csv',
    'eval': '/content/drive/MyDrive/NLP/data_sent/eval_subset.csv'
}, encoding='ISO-8859-1')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [166]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',                          # Directory for storing model checkpoints and evaluation results
    evaluation_strategy=IntervalStrategy.STEPS,      # Evaluate every specified number of steps
    save_strategy=IntervalStrategy.STEPS,            # Save the model every specified number of steps
    save_steps=500,                                  # Number of steps between two saves
    load_best_model_at_end=True,                     # Load the best model at the end of training
    num_train_epochs=4,                              # Number of training epochs
    per_device_train_batch_size=8,                   # Increase batch size if GPU memory allows
    per_device_eval_batch_size=8,                    # Increase batch size if GPU memory allows
    learning_rate=5e-5,                              # Adjust learning rate based on task and convergence
    weight_decay=0.01,                               # Weight decay for regularization
    warmup_steps=500,                                # Number of warmup steps
    logging_steps=100,                               # More frequent logging for quicker feedback
    fp16=True,                                       # Use 16-bit precision for faster training
    gradient_accumulation_steps=2,                   # Accumulate gradients over 2 steps for memory efficiency
    dataloader_num_workers=4,                        # Increase if CPU resources are available
    push_to_hub=True,                                # Push the model to the Hugging Face Model Hub
    hub_model_id="aaronayitey/Sentiment-classfication-ROBERTA-model",  # Model ID for pushing to the Hugging Face Model Hub
)

# Define the early stopping callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,                       # Stop training if no improvement after 3 evaluations
    early_stopping_threshold=0.005,                  # Set a small threshold for considering an improvement
)

# Combine the training arguments and the early stopping callback
training_args.callbacks = [early_stopping]


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [167]:
tokenizer_ROBERTA = AutoTokenizer.from_pretrained('roberta-base')

In [168]:
# Define a function to transform the label values
def transform_labels(label):
    # Extract the label value
    label = label['label']
    # Map the label value to an integer value
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2
    # Return a dictionary with a single key-value pair
    return {'labels': num}

# Define a function to tokenize the text data
def tokenize_data(example):
    # Extract the 'safe_text' value from the input example and tokenize it
    return tokenizer_ROBERTA(example['safe_text'], padding='max_length')

# Apply the transformation functions to the dataset using the 'map' method
# This transforms the label values and tokenizes the text data
dataset_out = dataset.map(transform_labels)

dataset_ROBERTA = dataset_out.map(tokenize_data, batched=True)

# Define a list of column names to remove from the dataset
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']

# Apply the 'transform_labels' function to the dataset to transform the label values
# Also remove the columns specified in 'remove_columns'

dataset_ROBERTA = dataset_ROBERTA.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/11781 [00:00<?, ? examples/s]

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

Map:   0%|          | 0/11781 [00:00<?, ? examples/s]

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

Map:   0%|          | 0/11781 [00:00<?, ? examples/s]

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

In [169]:
# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model_ROBERTA = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [170]:
train_dataset_ROBERTA = dataset_ROBERTA['train'].shuffle(seed=10)
eval_dataset_ROBERTA = dataset_ROBERTA['eval'].shuffle(seed=10)

In [171]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    return {"accuracy": accuracy}


In [172]:
trainer_ROBERTA = Trainer(
    model=model_ROBERTA,
    args=training_args,
    train_dataset=train_dataset_ROBERTA,
    eval_dataset=eval_dataset_ROBERTA,
    compute_metrics=compute_metrics    # Add this line to define the compute_metrics function
)

In [173]:
trainer_ROBERTA.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Step,Training Loss,Validation Loss,Accuracy
100,1.098500,1.068600,0.478276
200,0.875900,0.744133,0.678208
300,0.719700,0.867764,0.642227
400,0.720300,0.643429,0.735574
500,0.640800,0.622195,0.767481
600,0.608800,0.525827,0.807196
700,0.605800,0.564577,0.797692
800,0.498900,0.447011,0.845893
900,0.394600,0.482008,0.833333
1000,0.416500,0.383397,0.859470


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/util

Step,Training Loss,Validation Loss,Accuracy
100,1.098500,1.068600,0.478276
200,0.875900,0.744133,0.678208
300,0.719700,0.867764,0.642227
400,0.720300,0.643429,0.735574
500,0.640800,0.622195,0.767481
600,0.608800,0.525827,0.807196
700,0.605800,0.564577,0.797692
800,0.498900,0.447011,0.845893
900,0.394600,0.482008,0.833333
1000,0.416500,0.383397,0.859470


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/util

TrainOutput(global_step=2944, training_loss=0.35505365786831017, metrics={'train_runtime': 2828.2168, 'train_samples_per_second': 16.662, 'train_steps_per_second': 1.041, 'total_flos': 1.239132640790016e+16, 'train_loss': 0.35505365786831017, 'epoch': 4.0})

In [174]:
# Evaluate the model
eval_results = trainer_ROBERTA.evaluate()

# Create a dictionary of the evaluation results
results_dict = {
    "Model": "roberta-base",
    "Loss": eval_results["eval_loss"],
    "Accuracy": eval_results["eval_accuracy"],
    "Runtime": eval_results["eval_runtime"],
    "Samples Per Second": eval_results["eval_samples_per_second"],
    "Steps Per Second": eval_results["eval_steps_per_second"],
    "Epoch": eval_results["epoch"]
}

# Create a pandas DataFrame from the dictionary
results_df = pd.DataFrame([results_dict])

# Sort the results by "eval_accuracy" in descending order and get the name and state dict of the best model
best_model = results_df.loc[results_df['Accuracy'].idxmax()]

best_model

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Model                 roberta-base
Loss                      0.328185
Accuracy                  0.937203
Runtime                    38.6112
Samples Per Second          76.299
Steps Per Second             9.557
Epoch                          4.0
Name: 0, dtype: object

In [175]:
 # Push the final fine-tuned model to the Hugging Face model hub

trainer_ROBERTA.push_to_hub("aaronayitey/Sentiment-classfication-ROBERTA-model")
tokenizer_ROBERTA.push_to_hub("aaronayitey/Sentiment-classfication-ROBERTA-model")
model_ROBERTA.push_to_hub("aaronayitey/Sentiment-classfication-ROBERTA-model")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1699659850.3175c445c6ac.1182.4:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

events.out.tfevents.1699662763.3175c445c6ac.1182.5:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aaronayitey/Sentiment-classfication-ROBERTA-model/commit/70649b700fdfd10c120fa1e7b3d2513e7530d8cd', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='70649b700fdfd10c120fa1e7b3d2513e7530d8cd', pr_url=None, pr_revision=None, pr_num=None)

In [176]:
# Load the tokenizer
tokenizer = tokenizer_ROBERTA.from_pretrained("aaronayitey/Sentiment-classfication-ROBERTA-model")

# Load the fine-tuned model
model = pipeline("text-classification", model="aaronayitey/Sentiment-classfication-ROBERTA-model", tokenizer=tokenizer)


In [178]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Make predictions on some example text
result = model("this vaccine thing helps not.")

# Map the numerical label to the corresponding class name
result[0]["label"] = label_map[int(result[0]["label"].split("_")[1])]

# Print the predicted label and score
result

[{'label': 'negative', 'score': 0.9871618151664734}]